In [1]:
import numpy as np
import xsimlab as xs
#from scipy.integrate import odeint
import matplotlib.pyplot as plt

In [2]:
plt.rcParams['figure.figsize'] = [15, 10]

# OKAY!
so actually this higher order grid makes shit more complicated than it needs to be!
instead, keep track of component dims within component, keep state of components within components, BUT, with each added "box" or PhysEnv,
tha dimensionality of each Component changes, and stays coherent!

Fluxes can automatically be calcualted over all PhysEnvs/Boxes this way, vectorization..!



In [80]:
@xs.process
class HigherOrder:
    """so this keeps track of total dims, gives these to sub processes, which are components here"""
    dim = xs.variable(intent='in')
    H = xs.index(dims='H')
    
    def initialize(self):
        self.H = np.arange(self.dim)
    pass
    
@xs.process
class Comp:
    #H_dim = xs.foreign(HigherOrder,'dim')
    
            
    @xs.runtime(args="step_delta")
    def run_step(self, dt):
        print(self.label,'step')
        self.delta = sum((v for v in self.fluxes)) * dt  # multiply by time step
        print('delta', self.delta)
    
    @xs.runtime(args="step_delta")
    def finalize_step(self, dt):
        print(self.label,'fin')
        self.state += self.delta * self.state
        print('state', self.state)

@xs.process
class Nutrient(Comp):
    """so this keeps track of one type of component, state as numpy array 'inout', udpated here
    collects all fluxes acting on component, and provides states to fluxes to compute it
    
    importantly the components reflect the dimensionality of the full model! this way vectorisation takes care of all else (i hope)
    """
    label= xs.variable(default='N')
    dim = xs.variable(intent='in')
    N = xs.index(dims='N')
    
    state = xs.variable(dims=('H','N'), intent='inout')
    
    fluxes = xs.group('N_flux')
    
    #flowrate = xs.variable(dims=[(),('N'),('H','N')], intent='in')
    
    def initialize(self):
        self.N = np.arange(self.dim)
        print('N', self.N, self.state)

@xs.process
class Phytoplankton(Comp):
    """so this keeps track of one type of component, state as numpy array 'inout', udpated here
    collects all fluxes acting on component, and provides states to fluxes to compute it
    
    importantly the components reflect the dimensionality of the full model! this way vectorisation takes care of all else (i hope)
    """
    label= xs.variable(default='P')
    dim = xs.variable(intent='in')
    P = xs.index(dims='P')
    
    state = xs.variable(dims=('H','P'), intent='inout')
    
    fluxes = xs.group('P_flux')
    
    halfsat = xs.variable(dims=[(),('P'),('H','P')], intent='in')
    
    def initialize(self):
        self.P = np.arange(self.dim)
        print('P', self.P, self.state)


@xs.process
class Zooplankton(Comp):
    """so this keeps track of one type of component, state as numpy array 'inout', udpated here
    collects all fluxes acting on component, and provides states to fluxes to compute it
    
    importantly the components reflect the dimensionality of the full model! this way vectorisation takes care of all else (i hope)
    """    
    label= xs.variable(default='Z')
    dim = xs.variable(intent='in')
    Z = xs.index(dims='Z')
    
    state= xs.variable(dims=('H','Z'), intent='inout')
    
    fluxes = xs.group('Z_flux')
    
    def initialize(self):
        self.Z = np.arange(self.dim)
        print('Z',self.Z,self.state)


@xs.process
class Flux:
    """This is a flux, that collects component states and gives flux back to the components
    """
    N_flux = xs.variable(dims= ('H','N'), intent='out', groups='N_flux')
    P_flux = xs.variable(dims= ('H','P'), intent='out', groups='P_flux')
    Z_flux = xs.variable(dims= ('H','Z'), intent='out', groups='Z_flux')
    
    P_halfsat = xs.foreign(Phytoplankton, 'halfsat')
    
    N = xs.foreign(Nutrient, 'state')
    P = xs.foreign(Phytoplankton, 'state')
    Z = xs.foreign(Zooplankton, 'state')
    
    #@xs.runtime(args='step_delta')
    def run_step(self):
        P_nutlim = self.N/(self.P_halfsat + self.N) 
        print('nutlim',P_nutlim, np.sum(P_nutlim * self.P, axis = 1, keepdims = True))
        Z_mortality = 0.1
        
        self.N_flux = - np.sum(P_nutlim * self.P, axis = 1, keepdims = True)
        self.P_flux = P_nutlim * self.P
        
        self.Z_flux = - Z_mortality * self.Z
    

@xs.process
class ModelSetup:
    P_state = xs.foreign(Phytoplankton, 'state', intent='out')
    Z_state = xs.foreign(Zooplankton, 'state', intent='out')
    N_state = xs.foreign(Nutrient, 'state', intent='out')
    
    P_dim = xs.foreign(Phytoplankton, 'dim', intent='out')
    Z_dim = xs.foreign(Zooplankton, 'dim', intent='out')
    N_dim = xs.foreign(Nutrient, 'dim', intent='out')
    
    H_dim = xs.foreign(HigherOrder,'dim')
    
    dimP = xs.variable(intent='in')
    dimZ = xs.variable(intent='in')
    dimN = xs.variable(intent='in') 
    
    initValP = xs.variable(intent='in')
    initValZ = xs.variable(intent='in')
    initValN = xs.variable(intent='in')
    
    P_halfsat = xs.foreign(Phytoplankton, 'halfsat', intent='out')
    
    def initialize(self):
        self.P_dim = self.dimP
        self.Z_dim = self.dimZ
        self.N_dim = self.dimN
        self.P_state = np.full((self.H_dim,self.P_dim), self.initValP, dtype='float64')
        self.Z_state = np.full((self.H_dim,self.Z_dim), self.initValZ, dtype='float64')
        self.N_state = np.linspace(0.0001, 1, self.H_dim*self.N_dim).reshape(self.H_dim,self.N_dim)
        
        self.P_halfsat = np.linspace(0,1,self.H_dim * self.P_dim).reshape(self.H_dim,self.P_dim)

In [81]:
Simple = xs.Model({'H':HigherOrder, 'N':Nutrient, 'P':Phytoplankton, 'Z':Zooplankton, 'Flx':Flux, 'MS':ModelSetup})
Simple

<xsimlab.Model (6 processes, 10 inputs)>
H
    dim          [in]
MS
    dimZ         [in]
    initValZ     [in]
    dimN         [in]
    initValP     [in]
    initValN     [in]
    dimP         [in]
Flx
N
    label        [in]
P
    label        [in]
Z
    label        [in]

In [85]:
Simple_in = xs.create_setup(
    model=Simple,
    clocks={
         'time': np.linspace(0., 2., 10)
    },
    master_clock='time',
    input_vars={
        # set full dims
        'H__dim':5,
        
        'MS__dimN':1,
        'MS__initValN':5,
        
        'MS__dimP':2,
        'MS__initValP':1,
        
        'MS__dimZ':3,
        'MS__initValZ':2
        
    },
    output_vars={
        'N__state':'time',
        'P__state':'time',
        'Z__state':'time'
    }
)

In [86]:
with Simple:
    sim_out = Simple_in.xsimlab.run()

N [0] [[1.00000e-04]
 [2.50075e-01]
 [5.00050e-01]
 [7.50025e-01]
 [1.00000e+00]]
P [0 1] [[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Z [0 1 2] [[2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]]
nutlim [[1.00000000e+00 8.99190728e-04]
 [5.29486493e-01 4.28644889e-01]
 [5.29436677e-01 4.73709140e-01]
 [5.29420069e-01 4.90917421e-01]
 [5.29411765e-01 5.00000000e-01]] [[1.00089919]
 [0.95813138]
 [1.00314582]
 [1.02033749]
 [1.02941176]]
N step
delta [[-0.22242204]
 [-0.21291808]
 [-0.22292129]
 [-0.22674166]
 [-0.22875817]]
P step
delta [[2.22222222e-01 1.99820162e-04]
 [1.17663665e-01 9.52544197e-02]
 [1.17652595e-01 1.05268698e-01]
 [1.17648904e-01 1.09092760e-01]
 [1.17647059e-01 1.11111111e-01]]
Z step
delta [[-0.04444444 -0.04444444 -0.04444444]
 [-0.04444444 -0.04444444 -0.04444444]
 [-0.04444444 -0.04444444 -0.04444444]
 [-0.04444444 -0.04444444 -0.04444444]
 [-0.04444444 -0.04444444 -0.04444444]]
N fin
state [[7.77577958e-05]
 [1.96829510e-01]
 [3.88578208e-01]
 [5.79

In [87]:
sim_out

<xarray.Dataset>
Dimensions:       (H: 5, N: 1, P: 2, Z: 3, time: 10)
Coordinates:
  * H             (H) int64 0 1 2 3 4
  * N             (N) int64 0
  * P             (P) int64 0 1
  * Z             (Z) int64 0 1 2
  * time          (time) float64 0.0 0.2222 0.4444 0.6667 ... 1.556 1.778 2.0
Data variables:
    H__dim        int64 5
    MS__dimN      int64 1
    MS__dimP      int64 2
    MS__dimZ      int64 3
    MS__initValN  int64 5
    MS__initValP  int64 1
    MS__initValZ  int64 2
    N__label      <U1 'N'
    N__state      (time, H, N) float64 0.0001 0.2501 0.5 ... 0.0782 0.1137 0.149
    P__label      <U1 'P'
    P__state      (time, H, P) float64 1.0 1.0 1.0 1.0 ... 2.127 2.362 2.168
    Z__label      <U1 'Z'
    Z__state      (time, H, Z) float64 2.0 2.0 2.0 2.0 ... 1.413 1.413 1.413

# Today: ToDO

- write the basic components of phydra, using the above structure
- allow up to 3D dimensionality in basic construct! but ideally completely compatible down to 0D.. let's try!
actually i could keep all the dims, just have them at 0 dimensions. (but this needs to happen under the hood!)
- break it down into many small steps!


(C, x, y, z)
1. figure out what the final structure of a 3D numpy array containing all components looks like!

2. figure out how to deal with flexible dimensionality from the ground up!

3. create the basic matrix in some process, and then initialize all others with np.zeros_like() etc.


idea i had in bed: make dimensionality flexible through inheritance

BaseDim:
    (x,y,z)
    pass
    
EnvDim(BaseDim):
    (x,y,z,env)
    pass

CompDim(EnvDim):
    (x,y,z,env,comp)
    pass

In [98]:
#np.zeros((3,4,5,5))

In [111]:
# (x,y,z,c)

x = np.zeros((2,2,5))
x

array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]])

In [114]:
x[:,:,2] = 1
x

array([[[0., 1., 1., 0., 0.],
        [0., 1., 1., 0., 0.]],

       [[0., 1., 1., 0., 0.],
        [0., 1., 1., 0., 0.]]])

In [117]:
x.T[1]

array([[1., 1.],
       [1., 1.]])

In [ ]:
# The list of matrix computations

1. if I have a dimension in need of collapsing, e.g. to get all P biomass, this is done by choosing the correct axis for a sum function

2. np.transpose and np.swapaxes (for higher than 2D) are both useful for changing dimensionality (i assume this is useful when fluxes move between dimensions)

3. np.repeat if I want to fill up a matrix with a certain repeating set of values
3.1 can also use vstack and hstack to add boundaries, or special features

4. take slices and reshape for computing grazing etc. in a vectorized fashion!

5. boolean masking is always available, if I need a certain subset of the matrix

# a question i wanna check now, if there is an easy way to keep computations similar with a flexible number of higher dimensions
a) let's build an example N and an example P array
b) make them with (x,y,z) coordinates as well
c) compute a nutrient uptake flux on both
d) check if it works with any number of higher dimension!

the key is broadcasting!
this helps a lot.. nice

In [437]:
Pdim1=5
Ndim1=1

Env=3
x=2
y=2

In [438]:
np.linspace(0.1,1.,Env)

array([0.1 , 0.55, 1.  ])

In [439]:
P = np.tile(np.linspace(0.1,1.,Pdim1),(y,x,Env,1))
P

array([[[[0.1  , 0.325, 0.55 , 0.775, 1.   ],
         [0.1  , 0.325, 0.55 , 0.775, 1.   ],
         [0.1  , 0.325, 0.55 , 0.775, 1.   ]],

        [[0.1  , 0.325, 0.55 , 0.775, 1.   ],
         [0.1  , 0.325, 0.55 , 0.775, 1.   ],
         [0.1  , 0.325, 0.55 , 0.775, 1.   ]]],


       [[[0.1  , 0.325, 0.55 , 0.775, 1.   ],
         [0.1  , 0.325, 0.55 , 0.775, 1.   ],
         [0.1  , 0.325, 0.55 , 0.775, 1.   ]],

        [[0.1  , 0.325, 0.55 , 0.775, 1.   ],
         [0.1  , 0.325, 0.55 , 0.775, 1.   ],
         [0.1  , 0.325, 0.55 , 0.775, 1.   ]]]])

In [440]:
N = np.full((y,x,Env,Ndim1),5)
N[:,:,1]=3
N[:,:,2]=2
N

array([[[[5],
         [3],
         [2]],

        [[5],
         [3],
         [2]]],


       [[[5],
         [3],
         [2]],

        [[5],
         [3],
         [2]]]])

In [441]:
def nutlim(N,P,k_N):
    nlim = N/(k_N+N)
    return nlim*P

In [442]:
N

array([[[[5],
         [3],
         [2]],

        [[5],
         [3],
         [2]]],


       [[[5],
         [3],
         [2]],

        [[5],
         [3],
         [2]]]])

In [443]:
N_right

array([[[[3],
         [2],
         [2]],

        [[3],
         [2],
         [2]]]])

In [ ]:
def nfluxbetweenECOs(N):
    np.roll()
    return nfluxes

In [444]:
print(N)
N_left = np.roll(N, 1)
N_left[:,:,0] = 5
N_right = np.roll(N, -1)
N_right[:,:,-1] = 2
N_advected = 0.5 * (N_left+N_right) - 0.1 * (N_right-N_left)

N_mix = N_advected-N

N+N_mix

[[[[5]
   [3]
   [2]]

  [[5]
   [3]
   [2]]]


 [[[5]
   [3]
   [2]]

  [[5]
   [3]
   [2]]]]


array([[[[4.2],
         [3.8],
         [2.6]],

        [[4.2],
         [3.8],
         [2.6]]],


       [[[4.2],
         [3.8],
         [2.6]],

        [[4.2],
         [3.8],
         [2.6]]]])

In [ ]:
factor = self.v / (2 * self.grid_spacing)

u_left = np.roll(self.u, 1)
u_right = np.roll(self.u, -1)
        
u_1 = 0.5 * (u_right + u_left) - factor * dt * (u_right - u_left)

self.u_advected = u_1 - self.u

In [445]:
nlim_flux = nutlim(N,P,.5)
nlim_flux

array([[[[0.09090909, 0.29545455, 0.5       , 0.70454545, 0.90909091],
         [0.08571429, 0.27857143, 0.47142857, 0.66428571, 0.85714286],
         [0.08      , 0.26      , 0.44      , 0.62      , 0.8       ]],

        [[0.09090909, 0.29545455, 0.5       , 0.70454545, 0.90909091],
         [0.08571429, 0.27857143, 0.47142857, 0.66428571, 0.85714286],
         [0.08      , 0.26      , 0.44      , 0.62      , 0.8       ]]],


       [[[0.09090909, 0.29545455, 0.5       , 0.70454545, 0.90909091],
         [0.08571429, 0.27857143, 0.47142857, 0.66428571, 0.85714286],
         [0.08      , 0.26      , 0.44      , 0.62      , 0.8       ]],

        [[0.09090909, 0.29545455, 0.5       , 0.70454545, 0.90909091],
         [0.08571429, 0.27857143, 0.47142857, 0.66428571, 0.85714286],
         [0.08      , 0.26      , 0.44      , 0.62      , 0.8       ]]]])

In [446]:
P+nlim_flux

array([[[[0.19090909, 0.62045455, 1.05      , 1.47954545, 1.90909091],
         [0.18571429, 0.60357143, 1.02142857, 1.43928571, 1.85714286],
         [0.18      , 0.585     , 0.99      , 1.395     , 1.8       ]],

        [[0.19090909, 0.62045455, 1.05      , 1.47954545, 1.90909091],
         [0.18571429, 0.60357143, 1.02142857, 1.43928571, 1.85714286],
         [0.18      , 0.585     , 0.99      , 1.395     , 1.8       ]]],


       [[[0.19090909, 0.62045455, 1.05      , 1.47954545, 1.90909091],
         [0.18571429, 0.60357143, 1.02142857, 1.43928571, 1.85714286],
         [0.18      , 0.585     , 0.99      , 1.395     , 1.8       ]],

        [[0.19090909, 0.62045455, 1.05      , 1.47954545, 1.90909091],
         [0.18571429, 0.60357143, 1.02142857, 1.43928571, 1.85714286],
         [0.18      , 0.585     , 0.99      , 1.395     , 1.8       ]]]])

In [447]:
np.sum(nlim_flux,axis=3,keepdims=True)

array([[[[2.5       ],
         [2.35714286],
         [2.2       ]],

        [[2.5       ],
         [2.35714286],
         [2.2       ]]],


       [[[2.5       ],
         [2.35714286],
         [2.2       ]],

        [[2.5       ],
         [2.35714286],
         [2.2       ]]]])

In [448]:
x = N-np.sum(nlim_flux,axis=-1,keepdims=True)

In [449]:
x

array([[[[ 2.5       ],
         [ 0.64285714],
         [-0.2       ]],

        [[ 2.5       ],
         [ 0.64285714],
         [-0.2       ]]],


       [[[ 2.5       ],
         [ 0.64285714],
         [-0.2       ]],

        [[ 2.5       ],
         [ 0.64285714],
         [-0.2       ]]]])

# so the larger component can be computed, without transforming! but it should be dependent on Ndim and Pdim

AND I NEED TO GIVE GRAZING A QUICK TRY AS WELL LIKE THIS, with single N works a charm, but how to do pairwise calc?!


just need to make sure that 3D dims are defined at start.. let's give it a try if that works with inhertiance like I thought.. so that if it is not otherwise given, the higher dims are there, but collapsed.. don't make it any more difficult than it needs to be for model users & creators!

In [ ]:
#@xs.process Grid (Base Class!)

@xs.process
class GridX:
    dim_label = xs.variable(default='x')

    x = xs.index(dims='y')
    
    x_dim = xs.variable(intent='in')
    
    Env_dims = xs.group('Env_dim')
    
    #dims = xs.foreign(PhysicalEnvironment,'dims', intent='out')
    
    def initialize(self):
        #self.dims = (self.x_dim, self.Env_dim)
        print(list(self.Env_dims))
        self.x = np.array([f"{self.dim_label}-{i+1}" for i in range(self.x_dim)])
        

@xs.process
class GridXY:
    dim_label = xs.variable(default='y')
    
    y = xs.index(dims='y')
    
    y_dim = xs.variable(intent='in')
    
    Env_dim = xs.foreign(PhysicalEnvironment, 'Env_dim')
    x_dim = xs.foreign(GridX, 'x_dim')
    
    #dims = xs.foreign(PhysicalEnvironment,'dims', intent='out')

    def initialize(self):
        #self.dims = (self.y_dim, self.x_dim, self.Env_dim)        
        self.y = np.array([f"{self.dim_label}-{i+1}" for i in range(self.y_dim)])
        


In [205]:
@xs.process
class PhysicalEnvironment:
    dim_label = xs.variable(default='Env')
    
    Env = xs.index(dims='Env')
    
    # Input
    dim = xs.variable(intent='in',groups='Env_dim')
    
    def initialize(self):
        print('PhysEnv_Init')
        self.Env = np.array([f"{self.dim_label}-{i+1}" for i in range(self.dim)])
        print(self.Env)
        
    #def run_step(self):
    #    #print(self.dims)
    #    pass
    
@xs.process
class Grid1D:
    dim_label = xs.variable(default='x')
    
    index = xs.index(dims='x')
    
    def initialize(self):
        print('PhysEnv_Init')
        self.index = np.array([f"{self.dim_label}-{i+1}" for i in range(self.dim)])
        print(self.index)
        super(Grid1D, self).initialize()
        print(self.index)

        
@xs.process
class GridSetup:
    HigherDims = xs.foreign(PhysicalEnvironment,'dims')
    # two things I gotta do: keep track of all higher levels of dims above PhysEnv
    # Component's always belong in specific PhysEnv (inheritance, here?????)
    
# What do I actually want to do? I need to 
    # a) supply state arrays, in full dims, at simulation start
    # b) Allow multiple PhysEnvs? No just one for now! but what do I wnat to do.. 
# I want the PhysEnv to know all higher dims

# Question:
# - how do i set up 0D model? (i.e. one PhysEnv, e.g. slab)
# - how could I make it so that putting this 0D into 1D is straightforward?
# - 2d
# - 3d ?

# could the PhysEnv define additional dims, always? then all comps init, and 
# are repeated by whatever factor necessary for full dims

# but essentially this should be the job of a separate process, so that I could potentially modify and 
# change state at start specifically for some Envs of Comps...
        

@xs.process
class Component:
    Env_dim = xs.foreign(PhysicalEnvironment, 'dim')
    
    
    @xs.runtime(args="step_delta")
    def run_step(self, dt):
        print(self.dim_label,'step')
        self.delta = sum((v for v in self.fluxes)) * dt  # multiply by time step
        print('delta', self.delta)
    
    @xs.runtime(args="step_delta")
    def finalize_step(self, dt):
        print(self.dim_label,'fin')
        self.state += self.delta * self.state
        print('state', self.state)
    
    

@xs.process
class SingularComp(Component):
    dim = xs.variable(intent='out')
    #e.g. N
    
    def initialize(self):
        self.dim = 1

        
@xs.process
class Nutrient(SingularComp):
    # create the own N dimension
    dim_label = xs.variable(default='N')
    N = xs.index(dims='N')
    
    initval = xs.variable(intent='in', dims=[(),('N')])
    
    state = xs.variable(intent='out', dims=[('N'),('Env','N'),('x','Env','N'),('y','x','Env','N')])
    fluxes = xs.group('N_flux')
    
    def initialize(self):
        super(Nutrient, self).initialize()
        self.N = np.array([f"{self.dim_label}" for i in range(self.dim)])
        self.state = np.array([self.initval for i in range(self.dim)], dtype='float64')
        print('Nutrient', self.N, self.state)
        print('ENVDIM',self.Env_dim)
    
@xs.process
class MultiComp(Component):
    #e.g. P, Z 
    dim = xs.variable(intent='in')
    
    
@xs.process 
class Phytoplankton(MultiComp):
    dim_label = xs.variable(default='P')
    P = xs.index(dims='P')
    
    initval = xs.variable(intent='in', dims=[(),('P')])
    
    state = xs.variable(intent='out', dims=[('P'),('Env','P'),('x','Env','P'),('y','x','Env','P')])
    fluxes = xs.group('P_flux')
    
    def initialize(self):
        self.P = np.array([f"{self.dim_label}-{i+1}" for i in range(self.dim)])
        self.state = np.array([self.initval for i in range(self.dim)], dtype='float64')
        print('Phytoplankton', self.P, self.state)
        print('ENVDIM',self.Env_dim)



# Just build the fucking 3D model below here, come on!

In [378]:
@xs.process
class PhysicalEnvironment:
    dim_label = xs.variable(default='Env')
    x_label = xs.variable(default='x')
    y_label = xs.variable(default='y')
    
    Env = xs.index(dims='Env')
    x = xs.index(dims='x')
    y = xs.index(dims='y')
    
    # Input
    dims = xs.variable(intent='out')
    Env_dim = xs.variable(default=1)
    x_dim = xs.variable(intent='in')
    y_dim = xs.variable(intent='in')
    
    def initialize(self):
        print('PhysEnv_Init')
        self.Env = np.array([f"{self.dim_label}"])
        self.x = np.array([f"{self.x_label}-{i}" for i in range(self.x_dim)])
        self.y = np.array([f"{self.y_label}-{i}" for i in range(self.y_dim)])
        print(self.Env)
        self.dims = (self.y_dim, self.x_dim, self.Env_dim, 1)
        print(np.zeros(self.dims))


@xs.process
class Component:
    Model_dims = xs.foreign(PhysicalEnvironment, 'dims')
    
    @xs.runtime(args="step_delta")
    def run_step(self, dt):
        print(self.dim_label,'step')
        self.delta = sum((v for v in self.fluxes)) * dt  # multiply by time step
        print('delta', self.delta)
    
    @xs.runtime(args="step_delta")
    def finalize_step(self, dt):
        print(self.dim_label,'fin')
        self.state += self.delta * self.state
        print('state', self.state)
    
@xs.process
class SingularComp(Component):
    dim = xs.variable(intent='out')
    #e.g. N
    
    def initialize(self):
        self.dim = 1

        
@xs.process
class Nutrient(SingularComp):
    # create the own N dimension
    dim_label = xs.variable(default='N')
    N = xs.index(dims='N')
    
    initval = xs.variable(intent='in', dims=[(),('N')])
    
    state = xs.variable(intent='out', dims=[('N'),('Env','N'),('y','x','Env','N')])
    fluxes = xs.group('N_flux')
    
    def initialize(self):
        super(Nutrient, self).initialize()
        
        self.N = np.array([f"{self.dim_label}" for i in range(self.dim)])

        self.state = np.tile(np.array([self.initval], dtype='float64'),self.Model_dims)
        
        print('Nutrient', self.N, self.state.shape)
    
@xs.process
class MultiComp(Component):
    #e.g. P, Z 
    dim = xs.variable(intent='in')
    
    
@xs.process 
class Phytoplankton(MultiComp):
    dim_label = xs.variable(default='P')
    P = xs.index(dims='P')
    
    initval = xs.variable(intent='in', dims=[(),('P')])
    
    state = xs.variable(intent='out', dims=[('P'),('Env','P'),('y','x','Env','P')])
    fluxes = xs.group('P_flux')
    
    def initialize(self):
        self.P = np.array([f"{self.dim_label}-{i+1}" for i in range(self.dim)])
        
        self.state = np.tile(np.array([self.initval for i in range(self.dim)], dtype='float64'), self.Model_dims)
        
        print('Phytoplankton', self.P, self.state, self.state.shape)

In [379]:
DimModel = xs.Model({'N':Nutrient,'P':Phytoplankton,'PE':PhysicalEnvironment})#, 'GX':GridX, 'GXY':GridXY})

In [380]:
DimModel

<xsimlab.Model (3 processes, 11 inputs)>
PE
    dim_label     [in]
    Env_dim       [in]
    y_label       [in]
    y_dim         [in]
    x_label       [in]
    x_dim         [in]
N
    initval       [in] () or ('N',) 
    dim_label     [in]
P
    dim           [in]
    initval       [in] () or ('P',) 
    dim_label     [in]

In [381]:
DimModel_in = xs.create_setup(
    model=DimModel,
    clocks={
         'time': np.linspace(0., 2., 10)
    },
    master_clock='time',
    input_vars={
        'PE__x_dim':1,
        'PE__y_dim':1,
        
        'N__initval':2,
        
        'P__dim':2,
        'P__initval':1
        
    },
    output_vars={
        'N__state':'time',
        'P__state':'time'
    }
)

In [382]:
with DimModel:
    DimModel_out = DimModel_in.xsimlab.run()

PhysEnv_Init
['Env']
[[[[0.]]]]
Nutrient ['N'] (1, 1, 1, 1)
Phytoplankton ['P-1' 'P-2'] [[[[1. 1.]]]] (1, 1, 1, 2)
N step
delta 0.0
P step
delta 0.0
N fin
state [[[[2.]]]]
P fin
state [[[[1. 1.]]]]
N step
delta 0.0
P step
delta 0.0
N fin
state [[[[2.]]]]
P fin
state [[[[1. 1.]]]]
N step
delta 0.0
P step
delta 0.0
N fin
state [[[[2.]]]]
P fin
state [[[[1. 1.]]]]
N step
delta 0.0
P step
delta 0.0
N fin
state [[[[2.]]]]
P fin
state [[[[1. 1.]]]]
N step
delta 0.0
P step
delta 0.0
N fin
state [[[[2.]]]]
P fin
state [[[[1. 1.]]]]
N step
delta 0.0
P step
delta 0.0
N fin
state [[[[2.]]]]
P fin
state [[[[1. 1.]]]]
N step
delta 0.0
P step
delta 0.0
N fin
state [[[[2.]]]]
P fin
state [[[[1. 1.]]]]
N step
delta 0.0
P step
delta 0.0
N fin
state [[[[2.]]]]
P fin
state [[[[1. 1.]]]]
N step
delta 0.0
P step
delta 0.0
N fin
state [[[[2.]]]]
P fin
state [[[[1. 1.]]]]


# So right now I am stuck at a very basic issue, and that is how to dynamically assign dimensionality in the xsimlab framework

could it be that that is not possible? I can definitiely 

In [3]:
#n = 'N'

@xs.process
class TryThis:
    state=xs.variable(default=1,dims=('x','y','Eco'))